In [11]:
import numpy as numpy

DEFINITION OF THE CONSTANTS OF MIRAGE 3

<div style="text-align: center;">
    <img src="images/dm3.jpg" alt="MIRAGE III" width="600">
</div>


In [12]:
l_ref = 5.24             # reference lenght in meters
l_t   = (3/2) * l_ref    # total lenght in meters
m     = 8400             # mass in kg
c     = 0.52             # center of gravity position as a pourcentage of total lenght
S     = 34               # wings surface in m^2
r_g   = 2.65             # radius of gyration in m

OUR CHOOSEN OPERATING POINT WILL BE 16 : 
Mach 1.71
Alt  511 ft

<div style="text-align: center;">
    <img src="images/aircraft-aerodynamic-points.png" alt="Aerodynamical points" width="600">
</div>

We want to determine the equilibrium conditions around the operation point. We will use the model with a state space representation (A,B,C,D). We consider the following vectors : 

### State Vector


\begin{equation*}
X = \begin{pmatrix}
V ~
\gamma ~
\alpha ~
q ~
\theta~ 
z~
\end{pmatrix}
\end{equation*}


### Command Vector


\begin{equation*}
U = \begin{pmatrix}
\delta_m
\end{pmatrix}
\end{equation*}
According to the graphs : 

\begin{equation*}
C_{x0} = 0.03 
\end{equation*}

\begin{equation}
C_{z\alpha} = 2.23
\end{equation}

\begin{equation}
C_{\delta_m} = 0.38 ~~ [rad^{-1}]
\end{equation}

\begin{equation}
\delta_{m0} = -0.007 ~~ [rad]
\end{equation}

\begin{equation}
\alpha_0 = 0.01 ~~ [rad]
\end{equation}

\begin{equation}
f = 0.608 
\end{equation}

\begin{equation}
f_{\delta} = 0.9
\end{equation}

\begin{equation}
k = 0.48
\end{equation}

\begin{equation}
C_{mq} = -0.27 ~~ [s/rad]
\end{equation}

In [13]:
l_ref = 5.24             # reference lenght in meters
l_t   = (3/2) * l_ref    # total lenght in meters
m     = 8400             # mass in kg
c     = 0.52             # center of gravity position as a pourcentage of total lenght
S     = 34               # wings surface in m^2
r_g   = 2.65             # radius of gyration in m


g0       = 9.81
rho0     = 1.225
R_air    = 287.0
gamma_air= 1.4
T0       = 288.15
h_eq     = 511 * 0.3048
Mach_eq  = 1.71

In [14]:
# --- coefficients fournis ---
C_x0         = 0.03
k            = 0.48
C_z_alpha    = 2.23    # dCz/dalpha [per rad] (utilise la valeur que tu as écrite)
C_z_delta_m  = 0.38     # dCz/ddelta_m [per rad]
delta_m0     = -0.007   # δm initial (rad)
f            = 0.608
c            = 0.52
f_delta      = 0.9
alpha0       = 0.01     # alpha0 initial (rad)

# calculs géométriques / bras de levier
x_g    = -c * l_t            # position du centre de gravité (m)
x_f    = -f * l_t            # position d'action de la force (m)
X      = x_f - x_g          # bras de levier utilisé dans la formule
x_fdelta = -f_delta * l_t
Y      = x_fdelta - x_g

# itération / tolérances
eps = 1e-6
max_iter = 200

In [15]:
# import math
# import pandas as pd

# f = 0.61  # Corrected

# def isa_density(h, T0=288.15, p0=101325.0, L=0.0065, R=287.0, g=9.81):
#     """Simple ISA troposphere density (valid up to 11 km)."""
#     T = T0 - L * h
#     p = p0 * (T / T0) ** (g / (R * L))
#     rho = p / (R * T)
#     return rho, T


# def compute_equilibrium():
#     # atmosphere at altitude
#     rho, T = isa_density(h_eq, T0=T0, R=R_air, g=g0)
#     a = math.sqrt(gamma_air * R_air * T)   # speed of sound
#     V = Mach_eq * a
#     Q = 0.5 * rho * V**2

#     # initialization
#     alpha = 0.0
#     F_px = 0.0

#     history = []

#     for i in range(max_iter):
#         # 1) vertical force balance
#         C_z_eq = (m * g0 - F_px * math.sin(alpha)) / (Q * S)

#         # 2) drag model
#         print("DEBUG:", type(C_x0), C_x0, type(k), k, type(C_z_eq), C_z_eq)

#         C_x_eq = C_x0 + k * C_z_eq**2

#         # 3) Cx_delta_m depending on Cz
#         C_x_delta_m = 2.0 * k * C_z_eq * C_z_delta_m

#         # 4) equilibrium delta_m
#         numerator = C_x_eq * math.sin(alpha) + C_z_eq * math.cos(alpha)
#         denominator = C_x_delta_m * math.sin(alpha) + C_z_delta_m * math.cos(alpha)
#         if abs(denominator) < 1e-12:
#             raise ZeroDivisionError("Denominator near zero when computing delta_m")

#         delta_m_eq = delta_m0 - (numerator / denominator) * (X / (Y - X))

#         # 5) alpha update
#         alpha_next = alpha0 + (C_z_eq / C_z_alpha) - (C_z_delta_m / C_z_alpha) * delta_m_eq

#         # 6) new thrust axial
#         if abs(math.cos(alpha_next)) < 1e-6:
#             raise ValueError("cos(alpha_next) too small; invalid regime")

#         F_px_next = Q * S * C_x_eq / math.cos(alpha_next)

#         # save iteration
#         history.append({
#             'iter': i,
#             'alpha_rad': alpha,
#             'alpha_deg': math.degrees(alpha),
#             'Cz_eq': C_z_eq,
#             'Cx_eq': C_x_eq,
#             'delta_m_eq_rad': delta_m_eq,
#             'delta_m_eq_deg': math.degrees(delta_m_eq),
#             'F_px': F_px,
#         })

#         # convergence check
#         if abs(alpha_next - alpha) < eps:
#             alpha = alpha_next
#             F_px = F_px_next
#             break

#         alpha = alpha_next
#         F_px = F_px_next

#     else:
#         i = max_iter - 1

#     # pack all computed final values
#     result = {
#         'alpha_eq_rad': alpha,
#         'alpha_eq_deg': math.degrees(alpha),
#         'delta_m_eq_rad': delta_m_eq,
#         'delta_m_eq_deg': math.degrees(delta_m_eq),
#         'Cz_eq': C_z_eq,
#         'Cx_eq': C_x_eq,
#         'C_x_delta_m': C_x_delta_m,
#         'F_px': F_px,
#         'Q': Q,
#         'V': V,
#         'rho': rho,
#         'T': T,
#         'a': a,
#         'iterations': i + 1,
#         'converged': abs(alpha_next - alpha) < eps,
#         'X': X,
#         'Y': Y
#     }

#     df_hist = pd.DataFrame(history)
#     return result, df_hist


# # --- Example run ---
# res, hist = compute_equilibrium()

# print("Equilibrium result:")
# for k, v in res.items():
#     if isinstance(v, float):
#         print(f"  {k:15s} = {v:.6g}")
#     else:
#         print(f"  {k:15s} = {v}")

# display(hist.tail(8))


In [16]:
import math
import pandas as pd

def isa_density(h, T0=288.15, p0=101325.0, L=0.0065, R=287.0, g=9.81):
    T = T0 - L * h
    p = p0 * (T / T0) ** (g / (R * L))
    rho = p / (R * T)
    return rho, T


def compute_equilibrium():

    ## --- Atmosphère ---
    rho, T = isa_density(h_eq, T0=T0, R=R_air, g=g0)
    a = math.sqrt(gamma_air * R_air * T)
    V = Mach_eq * a
    Q = 0.5 * rho * V**2

    ## --- Init ---
    alpha = 0.0
    F_px = 0.0

    history = []

    for i in range(max_iter):

        C_z_eq = (m * g0 - F_px * math.sin(alpha)) / (Q * S)
        C_x_eq = C_x0 + k * C_z_eq**2
        C_x_delta_m = 2.0 * k * C_z_eq * C_z_delta_m

        numerator = C_x_eq * math.sin(alpha) + C_z_eq * math.cos(alpha)
        denominator = C_x_delta_m * math.sin(alpha) + C_z_delta_m * math.cos(alpha)

        if abs(denominator) < 1e-12:
            raise ZeroDivisionError("Invalid denominator computing delta_m.")

        delta_m_eq = delta_m0 - (numerator / denominator) * (X / (Y - X))

        alpha_next = alpha0 + (C_z_eq / C_z_alpha) - (C_z_delta_m / C_z_alpha) * delta_m_eq

        if abs(math.cos(alpha_next)) < 1e-6:
            raise ValueError("cos(alpha_next) too small.")

        F_px_next = Q * S * C_x_eq / math.cos(alpha_next)

        history.append((i, alpha, C_z_eq, C_x_eq, delta_m_eq, F_px))

        if abs(alpha_next - alpha) < eps:
            alpha = alpha_next
            F_px = F_px_next
            break

        alpha = alpha_next
        F_px = F_px_next

    else:
        i = max_iter - 1

    ## --- Return TOUTES les valeurs ---
    return (
        alpha,                     # 0
        math.degrees(alpha),       # 1
        delta_m_eq,                # 2
        math.degrees(delta_m_eq),  # 3
        C_z_eq,                    # 4
        C_x_eq,                    # 5
        C_x_delta_m,               # 6
        F_px,                      # 7
        Q,                         # 8
        V,                         # 9
        rho,                       # 10
        T,                         # 11
        a,                         # 12
        i + 1,                     # 13 (iterations)
        abs(alpha_next - alpha) < eps,  # 14 conversion flag
        X,                         # 15
        Y,                         # 16
        history                    # 17 (raw history)
    )


In [ ]:
(alpha,
 alpha_deg,
 delta_m,
 delta_m_deg,
 C_z_eq,
 C_x_eq,
 C_x_delta_m,
 F_p_x,
 Q,
 V,
 rho,
 T,
 a,
 iters,
 conv,
 X_val,
 Y_val,
 hist) = compute_equilibrium()


In [ ]:

# --- Créer un dictionnaire nom:valeur ---
variables = {
    "alpha (rad)": alpha,
    "alpha (deg)": alpha_deg,
    "delta_m (rad)": delta_m,
    "delta_m (deg)": delta_m_deg,
    "C_z_eq": C_z_eq,
    "C_x_eq": C_x_eq,
    "C_x_delta_m": C_x_delta_m,
    "F_p_x": F_p_x,
    "Q": Q,
    "V": V,
    "rho": rho,
    "T (K)": T,
    "a (m/s)": a,
    "iterations": iters,
    "converged": conv,
    "X": X_val,
    "Y": Y_val
}

# --- Convertir en DataFrame pour affichage joli tableau ---
df_vars = pd.DataFrame(list(variables.items()), columns=["Variable", "Valeur"])
df_vars


NameError: name 'Fpx' is not defined

In [ ]:
I_yy = m * Y**2
C_m_alpha = -C_z_alpha*(f-c)

In [ ]:
import math

def compute_longitudinal_forces(alpha_eq, delta_m_eq, V_eq, Q, S, Cx, Cz_alpha, Cz_delta_m, F_eq, F_tau, g0, gamma_eq):
    """
    Calcule les coefficients X et Z pour le modèle longitudinal simplifié.
    
    Retourne les valeurs dans l'ordre :
    X_v, X_alpha, X_delta_m, X_tau, X_gamma, Z_v, Z_alpha, Z_delta_m, Z_tau, Z_gamma
    """
    # --- composantes X ---
    X_alpha = (F_eq * math.sin(alpha_eq) + Q * S * Cx) / (m * V_eq)
    X_delta_m = Q * S * Cx_delta_m / (m * V_eq)
    X_tau = - F_tau * math.cos(alpha_eq) / (m * V_eq)
    X_v = 2 * Q * S * Cx / (m * V_eq)
    X_gamma = g0 * math.cos(gamma_eq) / V_eq

    # --- composantes Z ---
    Z_alpha = (F_eq * math.cos(alpha_eq) + Q * S * Cz_alpha) / (m * V_eq)
    Z_delta_m = Q * S * Cz_delta_m / (m * V_eq)
    Z_tau = F_tau * math.sin(alpha_eq) / (m * V_eq)
    Z_v = 2 * Q * S * Cz / (m * V_eq)
    Z_gamma = g0 * math.sin(gamma_eq) / V_eq
    
    m_alpha = (Q * S * l_ref * C_m_alpha) / I_yy 
    m_q     = (Q * S * l_ref**2 * C_m_q) /  I_yy

    # Retourne toutes les valeurs séparément
    return X_v, X_alpha, X_delta_m, X_tau, X_gamma, Z_v, Z_alpha, Z_delta_m, Z_tau, Z_gamma

# Exemple d'utilisation avec tes variables
X_v, X_alpha, X_delta_m, X_tau, X_gamma, Z_v, Z_alpha, Z_delta_m, Z_tau, Z_gamma = compute_longitudinal_forces(
    alpha_eq=alpha,
    delta_m_eq=delta_m,
    V_eq=V,
    Q=Q,
    S=S,
    Cx=Cx,
    Cz_alpha=C_z_alpha,
    Cz_delta_m=C_z_delta_m,
    F_eq=Fpx,
    F_tau=T,
    g0=g0,
    gamma_eq=0.0  # tu peux mettre l'angle gamma_eq de ton choix
)

print(X_v, X_alpha, X_delta_m, X_tau, X_gamma, Z_v, Z_alpha, Z_delta_m, Z_tau, Z_gamma)


0.08531940278369694 0.042740142430507624 0.006132036504697477 -5.885236278218914e-05 0.01688973721910515 0.033618621188034416 -3.1212724760302324 0.5391453934620586 1.109746443431257e-07 0.0


In [ ]:
import numpy as np 

A = np.array([[-X_v, -X_gamma, -X_alpha, 0, 0, 0],
              [Z_v, 0, Z_alpha, 0, 0, 0],
              [-Z_v, 0, -Z_gamma, 1, 0, 0],
              [0, 0, m_alpha, m_q, 0, 0],  
              [0, 0, 0, 1, 0, 0],
              [0, V, 0, 0, 0, 0]])  # V is defined; Veq should be V


NameError: name 'm_alpha' is not defined